# Harris 角点检测算法

Harris 角点检测算法是一种经典的角点检测方法，广泛用于图像处理和计算机视觉领域。该算法由 Chris Harris 和 Mike Stephens 于 1988 年提出，主要用于检测图像中的角点或特征点。角点通常是图像中具有显著变化的区域，适合用于特征匹配和跟踪。
## 利用 Sobel 算子计算梯度

Sobel 算子是一种用于边缘检测的离散微分算子，通过计算图像灰度值的近似梯度来突出图像中的边缘。Sobel 算子通过与图像进行卷积操作来计算图像的梯度，通常分为水平方向（x 方向）和垂直方向（y 方向）两个部分。

### Sobel 算子定义

Sobel 算子有两个 3x3 的卷积核，分别用于计算 x 方向和 y 方向的梯度：

- 水平方向（x 方向）的 Sobel 卷积核：
  $$
  G_x = \begin{bmatrix}
  -1 & 0 & 1 \\
  -2 & 0 & 2 \\
  -1 & 0 & 1
  \end{bmatrix}
  $$

- 垂直方向（y 方向）的 Sobel 卷积核：
  $$
  G_y = \begin{bmatrix}
  -1 & -2 & -1 \\
  0 & 0 & 0 \\
  1 & 2 & 1
  \end{bmatrix}
  $$

### 计算过程

1. **灰度图像**：
   - 首先，将输入的彩色图像转换为灰度图像，因为边缘检测通常在灰度图像上进行。

2. **卷积操作**：
   - 分别使用 Sobel 算子的两个卷积核与灰度图像进行卷积，得到 x 方向和 y 方向的梯度图像 $G_x$ 和 $G_y$。

3. **梯度幅值和方向**：
   - 计算梯度幅值（Gradient Magnitude）：
     $$
     G = \sqrt{G_x^2 + G_y^2}
     $$
   - 计算梯度方向（Gradient Direction）：
     $$
     \theta = \arctan\left(\frac{G_y}{G_x}\right)
     $$


## 算法原理

Harris 角点检测算法的基本思想是通过计算图像灰度值的变化，找到图像中局部区域内的显著变化点。具体步骤如下：

1. **图像梯度计算**：
   - 计算图像在 x 和 y 方向的梯度，得到梯度图像 $I_x$ 和 $I_y$。
   - 梯度的计算通常使用 Sobel 算子。

2. **结构张量矩阵**：
   - 在每个像素点处，计算结构张量矩阵 $M$：
     $$
     M = \begin{bmatrix}
     I_x^2 & I_x I_y \\
     I_x I_y & I_y^2
     \end{bmatrix}
     $$
   - 其中，$I_x^2$、$I_y^2$ 和 $I_x I_y$ 是在窗口大小内对梯度平方和梯度乘积进行平滑处理后的结果。

### 对梯度平方和梯度乘积进行平滑处理

在 Harris 角点检测算法中，对图像的梯度平方和梯度乘积进行平滑处理是一个重要步骤。平滑处理的目的是减少噪声干扰，并更准确地计算图像局部区域的变化特征。

首先，通过 Sobel 算子计算图像在 x 和 y 方向的梯度，分别得到梯度图像 $I_x$ 和 $I_y$。然后，计算以下三个值：

- $I_{xx} = I_x^2$：梯度在 x 方向的平方。
- $I_{yy} = I_y^2$：梯度在 y 方向的平方。
- $I_{xy} = I_x \cdot I_y$：梯度在 x 和 y 方向的乘积。

这些值表示图像在不同方向上的变化程度。


为了减少噪声的影响，需要对以上三个梯度值进行平滑处理。平滑处理通常通过高斯滤波器来完成。

#### 高斯滤波器

高斯滤波器是一种线性平滑滤波器，用于去除图像中的高频噪声，同时保留图像的低频特征。高斯滤波器的卷积核由高斯函数定义，其形式如下：

$$
G(x, y) = \frac{1}{2\pi\sigma^2} \exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right)
$$

其中，$\sigma$ 是高斯函数的标准差，决定了平滑的程度。

#### 平滑处理步骤

1. **计算梯度的平方和乘积**：
   - $I_{xx} = I_x^2$
   - $I_{yy} = I_y^2$
   - $I_{xy} = I_x \cdot I_y$

2. **应用高斯滤波器进行平滑处理**：
   - $S_{xx} = G(x, y) * I_{xx}$
   - $S_{yy} = G(x, y) * I_{yy}$
   - $S_{xy} = G(x, y) * I_{xy}$

   其中，$*$ 表示卷积操作，$G(x, y)$ 是高斯滤波器。

3. **角点响应函数**：
   - 计算角点响应函数 $R$：
     $$
     R = \text{det}(M) - k \cdot (\text{trace}(M))^2
     $$
   - 其中，$\text{det}(M)$ 是矩阵 $M$ 的行列式，$\text{trace}(M)$ 是矩阵 $M$ 的迹（即对角线元素之和），$k$ 是经验参数，通常取值在 0.04 到 0.06 之间。

4. **角点检测**：
   - 对角点响应函数 $R$ 进行阈值化处理，选取响应值大于阈值的点作为角点。
   - 通常还需要进行非极大值抑制，以去除局部区域内的非极大值点，确保角点的唯一性。

# cv2.cornerHarris() 函数解释

`cv2.cornerHarris()` 是 OpenCV 库中用于角点检测的函数，基于 Harris 角点检测算法。该算法用于检测图像中的角点，这些角点在图像处理和计算机视觉中通常用于特征检测和匹配。

## 函数定义

```python
cv2.cornerHarris(src, blockSize, ksize, k[, dst[, borderType]])
```

### 参数解释

- `src`：输入的灰度图像。
- `blockSize`：角点检测的邻域大小（窗口大小）。
- `ksize`：Sobel 算子窗口大小，用于计算图像的梯度。
- `k`：Harris 角点检测的自由参数，通常取值在 0.04 到 0.06 之间。
- `dst`（可选）：输出图像，通常与输入图像有相同的类型和大小。
- `borderType`（可选）：边界模式，用于指定如何处理图像边界。默认值为 `cv2.BORDER_DEFAULT`。

### 返回值

- 返回一个与输入图像 `src` 尺寸相同的数组，包含每个像素的 Harris 响应值。较高的响应值表示潜在的角点。

In [ ]:
import cv2 
import numpy as np

img = cv2.imread('photoOpencv/test_1.jpg')
print ('img.shape:',img.shape)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# gray = np.float32(gray)
dst = cv2.cornerHarris(gray, 2, 3, 0.04)
print ('dst.shape:',dst.shape)

In [ ]:
img[dst>0.01*dst.max()]=[0,0,255]
cv2.imshow('dst',img) 
cv2.waitKey(0) 
cv2.destroyAllWindows()
